In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import pytesseract
import sympy as sp
from PIL import Image, ImageOps, ImageFilter,ImageEnhance
import string
import requests
from io import BytesIO
import base64

# 调用 EDGE 浏览器
driver = webdriver.Edge()
url = "https://zhxy.jxyjxy.com:8078/lyuapServer/login?service=http://218.87.96.236:4106/shiro-cas"
driver.get(url)

# 等待页面加载完成
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'form')))

# 使用WebDriver的page_source属性获取页面源代码
page_source = driver.page_source

# 使用BeautifulSoup解析页面源代码
soup = BeautifulSoup(page_source, 'html.parser')

# 直接查找验证码图片元素
img = driver.find_element(By.CSS_SELECTOR, 'form img')
img_src = img.get_attribute('src')
print("Image source:", img_src)


def preprocess_image(image):
    # 调整大小
    image = image.resize((300, 100))
    # 灰度化
    image = ImageOps.grayscale(image)
    # 二值化
    threshold = 150  # 调整阈值
    image = image.point(lambda p: p > threshold and 255)

    # 去噪
    image = image.filter(ImageFilter.MedianFilter(size=3))

    # 增强对比度
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2.0)

    # 显示预处理后的图像（可选）
    #image.show()

    return image

# Set the path to the Tesseract executable (update this path according to your installation)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def recognize_and_calculate_result(img_src):
    
    base64_image=img_src.replace("data:image/png;base64,","")
    # 将图片数据读取为二进制流
    img_data =  base64.b64decode(base64_image)
    with open("temp_image.png", "wb") as img_file:
        img_file.write(img_data)
    # Open the image
    image = Image.open("temp_image.png")
    image.show()

    # Apply preprocessing (you can customize this based on your needs)
    image = preprocess_image(image)

    # Use Tesseract OCR to extract text from the image
    recognize_text = pytesseract.image_to_string(image, config='--psm 6 -c tessedit_char_whitelist=0123456789+/*=?')
    expression_text = recognize_text.replace("=?","")

    try:
        # Evaluate the expression using sympy
        result = sp.sympify(expression_text)
        return expression_text, result
    except sp.SympifyError:
        return expression_text, "Error: Unable to evaluate the expression"

if __name__ == "__main__":
    # Provide the path to the image containing the math expression

    expression, result = recognize_and_calculate_result(img_src)

    print("Math Expression:", expression)
    print("Result:", result)

In [ ]:
from PIL import Image
import base64
from io import BytesIO

# 图像数据
image_data = "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGQAAAAZCAIAAABIPBwcAAACbUlEQVR42sWYO07FQAxFZyv0iA2w%0ABSQKVoH0OhooKFgRomYLdLQshkgjGT9/ru84eUJykSgTZ3xy/UnG18/3tI/3z83ktG1LfsDio/Zz%0AoI3D9zed7Od1UVil8/Dq2BPnRcXV28whos6uUrCYTes1/5iJ/C2XhaVtPyxZ09g0uES67S0YPBQG%0Alt4ov5WeuHbCYsoughXy6gmb0QsJ90BeZCxZ+CMM0sSAH/B0f2es3Nbjy43xDNY/n27FmCYA3oGc%0Avl69ifHiSmHxGuZhbYwMrHkAeGlSGa+yIJhLmlSGLE3Dtlw1LPBitavJazISZFnYGtBcqWGR6emP%0ADZo1WCbC1a4xYZWJLAs2RtNCmnJqpOQXl7BwkpKwztIwrLVkBwzTMHOixcXDKjuP3Ht9esCWSaaE%0A9VfgM1h8TwQ1KytbOhkzE1i6YAHBrsLSlZ7U5sjkDagZCWg6wiuEpQv8vEUO/BN9afcFnhkJQa4I%0AKWYUPyvwnkKpshBxKS5NLduWhhXWe/77LBuMJikwwfowU1hYXKDA6ebIz5MgDfUanYzGSZmGxr/A%0AYhQwbYDhLRsCerCyR2QQw6lKYHm3PVggOn8phVWmoaDRrnWNBwnOJIvnZco8+YcHFy88yiJYYCTJ%0A8PtW6AfUci7xz8rGd0NqDyzTB8s+FsDCbz70ZUj1vmyzrPStMNsk0xP1Gl2zwLgQK4uZ48ka3xup%0A+Ykcfw8yvJjvDS+uwf9pKOs980O9fP9LP5jIPxAkrIxX3A3Jt0c2RzJgHhY5PTIDak9cY+k/JxZg%0AGTAT7SGwQFEr5wPAayx905fZyofdvqstLhwpM4H/AhbhqltMvE75AAAAAElFTkSuQmCC"
#去除换行符
image_data.strip()
# 移除数据头部
base64_data = image_data.replace("data:image/png;base64,", "")

# 添加正确的填充
missing_padding = len(base64_data) % 4
if missing_padding:
    base64_data += '=' * (4 - missing_padding)

# 解码 base64 数据
image_bytes = base64.b64decode(base64_data)

# 读取为图像对象
image = Image.open(BytesIO(image_bytes))

# 显示图像（可选）
image.show()


In [14]:
import base64

# 图像数据字符串
img_data = "iVBORw0KGgoAAAANSUhEUgAAAGQAAAAZCAIAAABIPBwcAAACbUlEQVR42sWYO07FQAxFZyv0iA2w%0ABSQKVoH0OhooKFgRomYLdLQshkgjGT9/ru84eUJykSgTZ3xy/UnG18/3tI/3z83ktG1LfsDio/Zz%0AoI3D9zed7Od1UVil8/Dq2BPnRcXV28whos6uUrCYTes1/5iJ/C2XhaVtPyxZ09g0uES67S0YPBQG%0Alt4ov5WeuHbCYsoughXy6gmb0QsJ90BeZCxZ+CMM0sSAH/B0f2es3Nbjy43xDNY/n27FmCYA3oGc%0Avl69ifHiSmHxGuZhbYwMrHkAeGlSGa+yIJhLmlSGLE3Dtlw1LPBitavJazISZFnYGtBcqWGR6emP%0ADZo1WCbC1a4xYZWJLAs2RtNCmnJqpOQXl7BwkpKwztIwrLVkBwzTMHOixcXDKjuP3Ht9esCWSaaE%0A9VfgM1h8TwQ1KytbOhkzE1i6YAHBrsLSlZ7U5sjkDagZCWg6wiuEpQv8vEUO/BN9afcFnhkJQa4I%0AKWYUPyvwnkKpshBxKS5NLduWhhXWe/77LBuMJikwwfowU1hYXKDA6ebIz5MgDfUanYzGSZmGxr/A%0AYhQwbYDhLRsCerCyR2QQw6lKYHm3PVggOn8phVWmoaDRrnWNBwnOJIvnZco8+YcHFy88yiJYYCTJ%0A8PtW6AfUci7xz8rGd0NqDyzTB8s+FsDCbz70ZUj1vmyzrPStMNsk0xP1Gl2zwLgQK4uZ48ka3xup%0A+Ykcfw8yvJjvDS+uwf9pKOs980O9fP9LP5jIPxAkrIxX3A3Jt0c2RzJgHhY5PTIDak9cY+k/JxZg%0AGTAT7SGwQFEr5wPAayx905fZyofdvqstLhwpM4H/AhbhqltMvE75AAAAAElFTkSuQmCC"

# 检查是否包含 URL 安全的字符集
is_url_safe = any(char in img_data for char in '-_')
if is_url_safe:
    print("Base64 data is URL safe.")
else:
    print("Base64 data is not URL safe.")

Base64 data is not URL safe.
